In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import joblib

# Загрузим обработанный датасет
data_path = 'data/processed/'  # Путь к папке с .txt-книгами
book_filenames = ['book1.txt', 'book2.txt', 'book3.txt']  # Пример списка файлов
corpus = []
for filename in book_filenames:
    with open(data_path + filename, 'r', encoding='utf-8') as file:
        text = file.read()
        corpus.append(text)

# Создадим токенизатор
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Преобразуем текст в последовательности чисел
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Подготовим данные для обучения
max_sequence_length = max([len(seq) for seq in input_sequences])
X = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
y = X[:, -1]
X = X[:, :-1]

# Создадим модель RNN
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Обучим модель
model.fit(X, y, epochs=10, verbose=1)

# Сохраним модель
joblib.dump(model, 'models/rnn_model/rnn_model.h5')

print("Модель RNN обучена и сохранена в 'models/rnn_model'.")
